In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [20]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 4), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, 1)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        x = x.reshape(-1, 4*2*7)
        x = torch.sigmoid(self.fc1(x))
        return x


net = EEGNet().cuda(0)
print(net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 4)).cuda(0))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())

tensor([[0.1106]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [26]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(int(len(X)/batch_size)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

In [48]:
# Load LVO dataset     
lvo = pd.read_csv('./data/df_onsite.csv')
lvo = lvo['lvo']


In [49]:
lvo=lvo.to_numpy().reshape(lvo.shape[0],-1)[:88]
# lvo = np.vstack(( lvo[:83], lvo[111:]))

In [50]:
lvo.shape

(88, 1)

In [51]:
lvo = np.float32(lvo)

In [14]:
#length = 42084
path = './data/ecg_clean/'
name = '_EEG_baseline_stroke_study_updated.csv'
store = None
count = 0
for i in range(1,84):
    eeg = None
    if len(str(i)) == 1:
        eeg = pd.read_csv(path+'00'+str(i)+name).to_numpy()
    else:
        eeg = pd.read_csv(path+'0'+str(i)+name).to_numpy()
    
    eeg = eeg[:120,1:]
    eeg = eeg.reshape(1, eeg.shape[0], eeg.shape[1])

    if i == 1:
        store = eeg
    else:
        store = np.vstack((store, eeg))

    count+=1 

for i in range(111, 116):
    eeg = pd.read_csv(path+str(i)+name).to_numpy()
    eeg = eeg[:120,1:]
    eeg = eeg.reshape(1, eeg.shape[0], eeg.shape[1])
    store = np.vstack((store, eeg))

    count +=1 

store = store.reshape(store.shape[0], 1, store.shape[1], store.shape[2])


In [22]:
store = np.float32(store)

In [52]:
batch_size = 32

for epoch in range(10):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(int(len(store)/batch_size)-1):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(store[s:e])
        labels = torch.FloatTensor(np.array([lvo[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        # running_loss += loss.data[0]
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, store, lvo, params))
    # print "Validation - ", evaluate(net, X_val, y_val, params)
    # print "Test - ", evaluate(net, X_test, y_test, params)


Epoch  0


ValueError: Using a target size (torch.Size([1, 32, 1])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.